In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import make_url
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore
import pandas as pd
import numpy as np


In [2]:
# load environment variables
load_dotenv()

# may need to change this to a different name
openai_api_key = os.getenv("OPENAI_API_KEY")
connection_string = os.getenv("DB_CONNECTION") 

url = make_url(connection_string)

In [3]:
# Set up the PGVectorStore with PostgreSQL connection parameters
# vector_store = PGVectorStore.from_params(
#     database= 'ai_advising_db',
#     host=url.host,
#     password=url.password,
#     port=url.port,
#     user=url.username,
#     table_name="drucker_llama_embedding",# table name 
#     embed_dim=3072,  # openai embedding dimension
#     hybrid_search=True, 
#     text_search_config="english",
#     hnsw_kwargs=None, # hnsw parameters
# )